In [7]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn import preprocessing
from sklearn.metrics import confusion_matrix, recall_score, precision_score
from sklearn.svm import OneClassSVM
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import IsolationForest

df = pd.read_csv('C:\\Users\\Samir Prasun\\Desktop\\Insight data science\\Samir\\data.txt', sep=" ", header=None)

df.columns = ['Date', 'Time', 'Room', 'Signal']
print (df.shape)
df.head()


(1136680, 4)


,Date,Time,Room,Signal
0,2014-08-10,07:51:35.663213,Office,ON
1,2014-08-10,07:51:36.789485,Office,OFF
2,2014-08-10,07:51:44.856168,Office,ON
3,2014-08-10,07:51:44.956600,Office,OFF
4,2014-08-10,07:51:46.910921,Office,ON


In [9]:
scaler = StandardScaler()


T=[]
df['time'] =df['Date']+' '+df['Time']
for i in range(len(df)):
    df.time[i]=np.datetime64(df.time[i])


t=np.datetime64('2014-08-10 07:50:00.000')
while (t<df.time[len(df)-1]):
    T.append(t)
    t=t+np.timedelta64(1,'D')
    
print(df.time[:5])

0    2014-08-10T07:51:35.663213
1    2014-08-10T07:51:36.789485
2    2014-08-10T07:51:44.856168
3    2014-08-10T07:51:44.956600
4    2014-08-10T07:51:46.910921
dtype: object


In [10]:
j=0
O, L, K, B, Be = [], [], [], [], []
for i in range(len(T)):
    a=0
    b=0
    be=0
    k=0
    l=0
    
    while df.time[j]<=T[i]:
        j=j+1
        
        if (df.Room.values[j-1]=='Office'):
            if (df.Signal.values[j-1]=='ON'):
                a=a+1
            else: n=1
                        
        elif (df.Room.values[j-1]=='Bathroom'):
            if (df.Signal.values[j-1]=='ON'):
                            b=b+1  
            else: n=1
                       
        elif df.Room.values[j-1]=='Kitchen':
            if df.Signal.values[j-1]=='ON':
                            k=k+1    
            else: n=1
        elif df.Room.values[j-1]=='LivingRoom':
            if df.Signal.values[j-1]=='ON':
                            l=l+1
            else: n=1
        elif df.Room.values[j-1]=='Bedroom':
            if df.Signal.values[j-1]=='ON':
                            be=be+1
            else: n=1
                        
        
    O.append(a)
    L.append(l)
    K.append(k)
    B.append(b) 
    Be.append(be)
                  
df_day=pd.DataFrame()
df_day['Date']=T
df_day['Office'] =O
df_day['cooking']=K
df_day['bathroom']=B
df_day['bedroom']=Be
df_day['Livingroom']=L
           
df_day.head()           
    
    


,Date,Office,cooking,bathroom,bedroom,Livingroom
0,2014-08-10 07:50:00,0,0,0,0,0
1,2014-08-11 07:50:00,766,966,1011,194,174
2,2014-08-12 07:50:00,76,697,569,339,583
3,2014-08-13 07:50:00,423,727,1063,252,115
4,2014-08-14 07:50:00,345,870,812,234,121


In [11]:
df1=pd.DataFrame()

outliers_fraction = 0.1
# train oneclassSVM 
def SVM (outliers_fraction, room):
    data=df_day[[room]]
    np_scaled = scaler.fit_transform(data)
    data = pd.DataFrame(np_scaled)
    model = OneClassSVM(nu=outliers_fraction, kernel="rbf", gamma=0.01)
    model.fit(data)

    Anomaly = pd.Series(model.predict(data))

    fig, ax = plt.subplots(figsize=(10,6))

    a = df_day.loc[Anomaly == -1, ['Date',room]] #anomaly

    ax.plot(df_day['Date'], df_day[room], color='blue')
    ax.scatter(a['Date'],a[room], color='red')
    plt.title(room)
    #plt.close(ax)
    plt.show()
    

from ipywidgets import interactive
df1=pd.DataFrame()
df1=df_day.drop(['Date'],axis=1)
kw=dict(outliers_fraction=(0.001,0.1),room=sorted(list(df1.columns)))
interactive_plot = interactive(SVM, **kw)
interactive_plot

interactive(children=(FloatSlider(value=0.0505, description='outliers_fraction', max=0.1, min=0.001), Dropdown…